# Import

In [41]:
import requests 
import tensorflow as tf
import numpy as np
import json
import pandas as pd
import base64
from sklearn.model_selection import train_test_split

# Data Loading

In [70]:
df = pd.read_csv('data/Student_performance_data.csv')

if 'StudentID' in df.columns:
    df.drop(columns=['StudentID'], inplace=True)

    df.to_csv('data/Student_performance_data.csv')

df.head()

,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


# Set Label Class

In [71]:
class_names = ["A", "B", "C", "D", "F"]

## Split into features and label

In [72]:
X = df.drop(columns=['GradeClass'])
y = df['GradeClass']

In [73]:
row = np.array(X.iloc[0].values)
label = y.iloc[0]

print('Test row: ', row, '\n')
print('Test label: ', label)

Test row:  [17.          1.          0.          2.         19.83372281  7.
  1.          2.          0.          0.          1.          0.
  2.92919559] 

Test label:  2.0


In [75]:
print(X.iloc[0])

Age                  17.000000
Gender                1.000000
Ethnicity             0.000000
ParentalEducation     2.000000
StudyTimeWeekly      19.833723
Absences              7.000000
Tutoring              1.000000
ParentalSupport       2.000000
Extracurricular       0.000000
Sports                0.000000
Music                 1.000000
Volunteering          0.000000
GPA                   2.929196
Name: 0, dtype: float64


In [74]:
columns_to_exclude = ['StudyTimeWeekly', 'GPA']

converted_row = []
for i, value in enumerate(row):
    column_name = X.columns[i]
    if column_name in columns_to_exclude:
        converted_row.append(value)  # Keep the original value
    else:
        converted_row.append(int(value))  # Convert to int

#converted_row = np.array(converted_row)

In [76]:
def create_tf_example(row):
    feature = {
        'Age': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[0]])),
        'Gender': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[1]])),
        'Ethnicity': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[2]])),
        'ParentalEducation': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[3]])),
        'StudyTimeWeekly': tf.train.Feature(float_list=tf.train.FloatList(value=[row[4]])),
        'Absences': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[5]])),
        'Tutoring': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[6]])),
        'ParentalSupport': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[7]])),
        'Extracurricular': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[8]])),
        'Sports': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[9]])),
        'Music': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[10]])),
        'Volunteering': tf.train.Feature(int64_list=tf.train.Int64List(value=[row[11]])),
        'GPA': tf.train.Feature(float_list=tf.train.FloatList(value=[row[12]])),
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

In [77]:
serialized_example = create_tf_example(converted_row)

# Make Prediction Request

In [80]:
json_data = json.dumps(
    {
        "instances": [{"examples": {"b64": base64.b64encode(serialized_example).decode('utf-8')}}]
    }
)

endpoint = "http://localhost:8080/v1/models/student-performance-model:predict"

response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})

prediction = np.argmax(response.json()["predictions"][0])
print("The student grade is: ", class_names[prediction])

The student grade is:  C
